# [ATL14](https://nsidc.org/data/atl14/)


## Downloading data using earthaccess

In [ ]:
import earthaccess

auth = earthaccess.login()
earthaccess.__version__

In [ ]:
temporal_range = ("2021-01", "2023-01")
max_files = 10

download_list = []

results = earthaccess.search_data(short_name="ATL14",
                                  cloud_hosted=True,
                                  temporal = temporal_range,
                                  count=max_files)
for granule in results:
    if granule.size() < 500:
        download_list.append(granule)
        
print(f"total granules listed under 500MB: {len(download_list)}")
files = earthaccess.download(download_list, local_path="../data/ATL14")

## Loading data

In [ ]:
import xarray as xr
import hvplot.xarray
from cartopy import crs
import pathlib

gv.extension('bokeh', 'matplotlib')

In [ ]:
files = []
for file in pathlib.Path('../data/ATL14').glob('*.nc'):
    files.append(str(file))
files

## Exploring ATL16 with xarray

* A) Opening a single granule
> xarray uses h5py or netcdf4 to open `.h5` and `.nc` files, it opens the file and based on the signature it selects the backend
> If the files are local this should be pretty straight-forward. 
* B) Opening multiple granules and concatenating them by time (start or end) 
> Because this dataset does not have a time dimensionwe need to explicit include it if we want to concatenate mutiple granules
> We can do this using a preprocessing function

In [ ]:
# Single granule, xarray opens hdf5 files using the netcdf4/h5py libraries.
ds = xr.open_dataset(files[1])
ds

In [ ]:
import pyproj
from shapely.geometry import box
from shapely.ops import transform

project = pyproj.Transformer.from_proj(
    pyproj.Proj(init='epsg:3413'), # source coordinate system
    pyproj.Proj(init='epsg:4326'))

b = box(ds.x.min(), ds.y.min(), ds.x.max(), ds.y.max())

b2 = transform(project.transform, b)
b2

In [ ]:
b2.bounds

In [ ]:
ds.h.hvplot(
    'x', 'y',
    crs=crs.NorthPolarStereo(central_longitude=-45, true_scale_latitude=70),
    projection=crs.NorthPolarStereo(),
    frame_height=540,
    frame_width=800,
    cmap='viridis',
    project=False,
    rasterize=True,
    coastline=True
)

## Validating 

In [ ]:
## TODO